# This code to generate answer and with Colbert retriever for Hallubench dataset using Simple RAG pipeline with Mistral locally using Langchain Ollama

In [ ]:
#@title { vertical-output: true}
# Install ollama: The Ollama API is now available at 127.0.0.1:11434
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#@title { vertical-output: true}
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the Mistral model
!ollama pull mistral

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e

In [ ]:
#@title { vertical-output: true}
!pip install -q langchain
!pip install -U -q langchain-community
!pip install -U langchain-huggingface sentence-transformers
!pip install -q chromadb
!pip install datasets
!pip install -q -U ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/6

In [ ]:
#@title { vertical-output: true}
import pandas as pd
from datasets import Dataset, Features, Sequence, Value
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from pydantic import BaseModel, Field
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from tqdm import tqdm

# Initialize Ollama LLM and Embeddings
llm: BaseLanguageModel = Ollama(model="mistral", temperature = 0.1)

<ipython-input-4-c559dc418397>:18: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm: BaseLanguageModel = Ollama(model="mistral", temperature = 0.1)


In [ ]:
#@title { vertical-output: true}
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

<ipython-input-5-f0e2007b65f4>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#@title { vertical-output: true}
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [ ]:
#@title { vertical-output: true}
# Load the dataset from a local CSV file

df = pd.read_csv('/content/halubench_halueval_pass_3.csv')

# Ensure the dataset has the required columns: 'question', 'contexts', 'ground_truth'
required_columns = ['question', 'contexts', 'ground_truth']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"The dataset must contain these columns: {required_columns}")


In [ ]:
#@title { vertical-output: true}
def calculation(row, llm=llm, embed_model=embed_model, index_counter=[0]):
    try:
        # Split the context into chunks
        # Convert 'contexts' to string if it's not
        context = str(row['contexts'])
        question = str(row['question'])

        # Create a unique index name for each row
        index_name = f"cell_{index_counter[0]}"
        index_counter[0] += 1

        # Create an index
        RAG.index(
            collection=[context],
            index_name=index_name,
            max_document_length=512,
            split_documents=True,
        )

        # Define the prompt template
        prompt_template = """
        You are a helpful AI assistant. Use the following pieces of information to answer the user's question. Provide a short answer only, no explanation. If you don't know the answer, just say that you don't know, don't try to make up an answer.
        Context: {context}
        Question: {question}
        """

        prompt = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )

        # Create the RAG pipeline
        rag_pipeline = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=RAG.as_langchain_retriever(k=3),
            return_source_documents=True,
            chain_type_kwargs={'prompt': prompt}
        )
        result = rag_pipeline.invoke({"query": question})

        # Concatenate retrieved documents
        retrieved_docs = RAG.search(query={"query": question}, k=3)

        return pd.Series({
            'answer': result['result'],
            'retrieved_documents': retrieved_docs
        })

    except Exception as e:
        # If an error occurs, print it and return empty results
        print(f"Error processing row {index_counter[0]}: {str(e)}")
        return pd.Series({
            'answer': None,
            'retrieved_documents': None
        })

# Apply the calculation function to each row with a progress bar
tqdm.pandas(desc="Processing rows")
df[['answer', 'retrieved_documents']] = df.progress_apply(calculation, axis=1)

print("Processing complete!")

In [ ]:
#@title { vertical-output: true}
def calculation(row, llm=llm, embed_model=embed_model, index_counter=[0]):
    # Split the context into chunks
    # Convert 'contexts' to string if it's not
    context = str(row['contexts'])  # or row['contexts'].astype(str) if using pandas
    question = str(row['question'])

    # Create a unique index name for each row
    index_name = f"line_{index_counter[0]}"
    index_counter[0] += 1

    # Create an index
    RAG.index(
      collection=[context],
      index_name=index_name,
      max_document_length=512,
      split_documents=True,
    )

    # Define the prompt template
    prompt_template = """
    You are a helpful AI assistant. Use the following pieces of information to answer the user's question. Provide a short answer only, no explaination. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context: {context}
    Question: {question}
    """

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Create the RAG pipeline
    rag_pipeline = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=RAG.as_langchain_retriever(k=3),
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt}
    )
    result = rag_pipeline.invoke({"query": str(row['question'])})

    # Concatenate retrieved documents
    retrieved_docs = RAG.search(query={"query": str(row['question'])}, k=3)

    # Return the result and retrieved documents
    return pd.Series({
        'answer': result['result'],
        'retrieved_documents': retrieved_docs
    })

# Apply the calculation function to each row with a progress bar
tqdm.pandas(desc="Processing rows")
df[['answer', 'retrieved_documents']] = df.progress_apply(calculation, axis=1)

print("Processing complete!")

In [ ]:
# Select required columns for the Hugging Face dataset
df_hf = df[required_columns + ['answer', 'retrieved_documents']]

# Define the schema for the Hugging Face dataset
features = Features({
    'question': Value('string'),
    'contexts': Sequence(Value('string')),
    'ground_truth': Value('string'),
    'answer': Value('string'),
    'retrieved_documents': Value('string'),
})

In [ ]:
# Save the DataFrame with generated answers to a CSV file
output_csv_path = 'halubench_halueval_EnhancedRAG_pass_3.csv'
df.to_csv(output_csv_path, index=False)

In [ ]:

import google.colab

# Import necessary libraries
import os
from google.colab import files

# Specify the path to your CSV file in Colab
csv_file_path = '/content/halubench_halueval_EnhancedRAG_pass_3.csv'

# Check if the file exists
if os.path.exists(csv_file_path):
    # Download the file
    files.download(csv_file_path)
    print(f"File '{csv_file_path}' has been downloaded.")
else:
    print(f"Error: The file '{csv_file_path}' does not exist.")

import time

print("Runtime will disconnect in 5 minutes...")
# Wait for 5 minutes (300 seconds)
time.sleep(300)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File '/content/halubench_halueval_EnhancedRAG_pass_6.csv' has been downloaded.
Runtime will disconnect in 5 minutes...


In [ ]:
import google.colab
from google.colab import runtime
import time

print("Runtime will disconnect in 5 minutes...")
# Wait for 5 minutes (300 seconds)
time.sleep(300)

print("Disconnecting and deleting runtime...")
runtime.unassign()
print("Done. You can close this tab now.")